In [1]:
import gurobipy as gp
from gurobipy import GRB

# Define data
# Orders: k = 1 to 4
orders = {
    1: {
        'location': 'Lozan',
        'ready': 1,          # June 25
        'deadline': 4,       # June 28
        'weight': 180,
        'external_cost': 500,
        'external_duration': 2
    },
    2: {
        'location': 'Milan',
        'ready': 2,          # June 26
        'deadline': 5,       # June 29
        'weight': 30,
        'external_cost': 600,
        'external_duration': 4
    },
    3: {
        'location': 'Marsilya',
        'ready': 1,          # June 25
        'deadline': 6,       # June 30
        'weight': 100,
        'external_cost': 700,
        'external_duration': 2
    },
    4: {
        'location': 'Lyon',
        'ready': 3,          # June 27
        'deadline': 6,       # June 30
        'weight': 50,
        'external_cost': 600,
        'external_duration': 1
    },
}

# Fleet Truck Routes: r = 1 to 5
routes = {
    1: {'description': 'Milan-Lozan', 'duration': 1, 'cost': 800},
    2: {'description': 'Milan-Lyon', 'duration': 1, 'cost': 750},
    3: {'description': 'Milan-Lozan-Lyon', 'duration': 2, 'cost': 830},
    4: {'description': 'Marsilya-Lyon', 'duration': 2, 'cost': 450},
    5: {'description': 'Marsilya-Lozan', 'duration': 2, 'cost': 450},
}

# Vehicle Capacity
C = 250

# External Shipper Feasibility
# Determine for each order if external shipper can deliver on time
external_feasibility = {}
for k, v in orders.items():
    external_feasibility[k] = (v['ready'] + v['external_duration'] <= v['deadline'])

# Time horizon: days 1 to 6 (June 25 to June 30)
T = range(1, 7)  # Days 1 to 6

# Penalty is removed as there are no undelivered orders

# Determine applicable routes for each order
applicable_routes = {
    1: [1, 3, 5],  # Lozan
    2: [1, 2, 3],  # Milan
    3: [4, 5],     # Marsilya
    4: [2, 3, 4],  # Lyon
}


In [2]:

# Create model
model = gp.Model("Shipment_Consolidation_Dual_Mode")

# Add variables
# x[k,r,t]: Assign order k to fleet truck route r on day t
x = {}
for k in orders:
    for r in applicable_routes[k]:
        duration = routes[r]['duration']
        earliest_depart = orders[k]['ready'] + 1  # Shipment available one day after ready
        latest_depart = orders[k]['deadline'] - routes[r]['duration']
        feasible_days = [t for t in T if earliest_depart <= t <= latest_depart]
        for t in feasible_days:
            x[k, r, t] = model.addVar(vtype=GRB.BINARY, name=f"x_{k}_{r}_{t}")

# y[r,t]: Fleet truck route r departs on day t
y = {}
for r in routes:
    for t in T:
        y[r, t] = model.addVar(vtype=GRB.BINARY, name=f"y_{r}_{t}")

# z[k]: Assign order k to external shipper
z = {}
for k in orders:
    if external_feasibility[k]:
        z[k] = model.addVar(vtype=GRB.BINARY, name=f"z_{k}")
    else:
        # External shipper cannot deliver on time; set z[k] = 0
        z[k] = model.addVar(vtype=GRB.BINARY, name=f"z_{k}")
        model.addConstr(z[k] == 0, name=f"ExternalShipper_Infeasible_Order_{k}")

# Set objective: Minimize fleet truck costs + external shipper costs
fleet_truck_cost = gp.quicksum(routes[r]['cost'] * y[r, t] for r in routes for t in T)
external_shipper_cost = gp.quicksum(orders[k]['external_cost'] * z[k] for k in orders)

model.setObjective(fleet_truck_cost + external_shipper_cost, GRB.MINIMIZE)

# Add constraints

# 1. Assignment Constraints
for k in orders:
    assignment = gp.quicksum(x[k, r, t] for r in applicable_routes[k]
                             for t in T
                             if (k, r, t) in x)
    model.addConstr(assignment + z[k] == 1, name=f"Assignment_Order_{k}")

# 2. Capacity Constraints
for r in routes:
    for t in T:
        total_weight = gp.quicksum(orders[k]['weight'] * x[k, r, t]
                                   for k in orders
                                   if (r in applicable_routes[k] and
                                       (k, r, t) in x))
        model.addConstr(total_weight <= C * y[r, t], name=f"Capacity_Route_{r}_Day_{t}")

# 3. Linking Constraints: If an order is assigned to a fleet truck route, activate the route departure
for k in orders:
    for r in applicable_routes[k]:
        for t in T:
            if (k, r, t) in x:
                model.addConstr(x[k, r, t] <= y[r, t], name=f"Link_x_y_Order_{k}_Route_{r}_Day_{t}")

# Optimize model
model.optimize()

# Check if a feasible solution was found
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:\n")
    
    # Print y variables (fleet truck route departures)
    print("Fleet Truck Route Departures:")
    for r in routes:
        for t in T:
            if y[r, t].X > 0.5:
                departure_date = 24 + t  # June 25 is day 1
                print(f"  Route {r} ({routes[r]['description']}) departs on Day {t} (June {departure_date}) with Cost €{routes[r]['cost']}")
    
    print("\nOrder Assignments:")
    for k in orders:
        # Check if order is assigned to external shipper
        if z[k].X > 0.5:
            arrival_day = orders[k]['ready'] + orders[k]['external_duration']
            arrival_date = 24 + arrival_day
            departure_day = orders[k]['ready'] + 1
            departure_date = 24 + departure_day
            print(f"  Order {k} to {orders[k]['location']} assigned to External Shipper departing on Day {departure_day} (June {departure_date}) arriving on Day {arrival_day} (June {arrival_date}) with Cost €{orders[k]['external_cost']}")
        else:
            # Assigned to a fleet truck route
            assigned = False
            for r in applicable_routes[k]:
                for t in T:
                    if (k, r, t) in x and x[k, r, t].X > 0.5:
                        arrival_day = t + routes[r]['duration']
                        departure_date = 24 + t
                        arrival_date = 24 + arrival_day
                        print(f"  Order {k} to {orders[k]['location']} assigned to Route {r} ({routes[r]['description']}) departing on Day {t} (June {departure_date}) arriving on Day {arrival_day} (June {arrival_date}) with Cost €{routes[r]['cost']}")
                        assigned = True
                        break
                if assigned:
                    break
            if not assigned and z[k].X <= 0.5:
                # This should not happen due to constraints
                print(f"  Order {k} to {orders[k]['location']} has an undefined status.")
else:
    print("No optimal solution found.")


Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-09
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 54 rows, 53 columns and 111 nonzeros
Model fingerprint: 0x5c7aea86
Variable types: 0 continuous, 53 integer (53 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+02]
  Objective range  [5e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2600.0000000
Presolve removed 54 rows and 53 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 1250 2600 

Optimal solution found (tolerance 1.00e-04)
Best objective 

In [8]:
# --- 1. Define Graph Data ---
# List (or dict) of arcs: each tuple is (start_node, end_node, cost)
arcs = [
    (1, 2, 6),
    (3, 1, 9),
    (2, 5, 7),
    (2, 4, 11),
    (4, 2, 4),
    (4, 3, 3),
    (3, 6, 7),
    (6, 4, 8),
    (5, 6, 9),
    (5, 7, 10),
    (6, 5, 8),
    (8, 6, 5),
    (7, 8, 7),
    (8, 7, 6)
]
def solve_directed_chinese_postman(arcs):

    
    # Collect sets of nodes from arcs
    # V is simply the unique set of nodes
    nodes = set()
    for i,j,c in arcs:
        nodes.add(i)
        nodes.add(j)
    nodes = sorted(list(nodes))  # for consistent ordering

    # --- 2. Build the Model ---
    model = gp.Model("Chinese_Postman_Directed")

    # 2.1 Create decision variables x_ij
    # x_ij: how many times arc (i->j) is traversed
    x_vars = {}
    for (i, j, cost) in arcs:
        # We'll allow x_ij to be integer >= 1, but we place a lower bound constraint separately.
        x_vars[(i, j)] = model.addVar(vtype=GRB.INTEGER, lb=0, name=f"x_{i}_{j}")

    model.update()

    # 2.2 Objective Function: Min sum of costs * x_ij
    model.setObjective(
        gp.quicksum(cost * x_vars[(i, j)] for (i, j, cost) in arcs),
        GRB.MINIMIZE
    )

    # 2.3 Constraints

    # (a) At least once constraint:
    # x_ij >= 1 for every arc in original graph
    for (i, j, cost) in arcs:
        model.addConstr(x_vars[(i, j)] >= 1, name=f"arc_used_once_{i}_{j}")

    # (b) Flow balance: For each node v, sum in-flows = sum out-flows
    for v in nodes:
        in_flow = gp.quicksum(x_vars[(i, j)] for (i, j, c) in arcs if j == v)
        out_flow = gp.quicksum(x_vars[(i, j)] for (i, j, c) in arcs if i == v)
        model.addConstr(in_flow == out_flow, name=f"flow_balance_{v}")


    # --- 3. Solve the Model ---
    model.optimize()

    # --- 4. Retrieve the solution ---
    if model.status == GRB.OPTIMAL:
        print("\nOPTIMAL SOLUTION FOUND\n")
        print(f"Total Cost = {model.objVal}")

        # Print how many times each arc is used
        for (i, j, cost) in arcs:
            usage = x_vars[(i, j)].X
            if usage > 0.5:  # if used
                print(f"Arc ({i} -> {j}) used {int(usage)} time(s), cost per use = {cost}")
        
        model.write("Chinese_Postman_Directed.lp")
    else:
        print("No optimal solution found (status = {})".format(model.status))

if __name__ == "__main__":
    solve_directed_chinese_postman(arcs)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]


Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 14 columns and 42 nonzeros
Model fingerprint: 0x121b7f52
Variable types: 0 continuous, 14 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 118.0000000
Presolve removed 22 rows and 14 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 118 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.180000000000e+02, best bound 1.180000000000e+02, gap 0.0000%

OPTIMAL SOLUTION FOUND

Total Cost = 118.0
Arc (1 -> 2) used 1 time(s), cost per use = 6
Arc (3 -> 1) used 1 time(s), cost per use = 9
Arc (2 -> 5) used 1 time(s), cost per use = 7
Arc (2 -> 4) used 1 time

In [11]:
# --- 1. Define Graph Data ---

# required_arcs = list of (i, j, cost)
# optional_arcs = list of (i, j, cost)

required_arcs = [
    # (start_node, end_node, cost)
    (1, 2, 6),
    (3, 1, 9),
    (2, 5, 7),
    (6, 4, 8),
    (7, 8, 7)
]
optional_arcs = [
    # (start_node, end_node, cost)
    (4, 2, 4),
    (4, 3, 3),
    (2, 4, 11),
    (3, 6, 7),
    (6, 7, 6),
    (8, 6, 5),
    (5, 7, 10),
    (5, 6, 9),
    (8, 7, 6),
    (6, 5, 8)

]

def solve_directed_rural_postman(required_arcs, optional_arcs):

    # Combine them
    arcs = required_arcs + optional_arcs
    
    # Build sets of nodes
    nodes = set()
    for (i, j, c) in arcs:
        nodes.add(i)
        nodes.add(j)
    nodes = sorted(list(nodes))
    
    # --- 2. Build the Model ---
    model = gp.Model("Directed_Rural_Postman")
    
    # x[i,j] = number of times arc (i->j) is traversed
    x_vars = {}
    # For required arcs, we will enforce x_ij >= 1
    # For optional arcs, x_ij >= 0
    # We'll store all arcs in a single dictionary
    for (i, j, cost) in arcs:
        # Create an integer variable with a lower bound of 0
        x_vars[(i, j)] = model.addVar(
            vtype=GRB.INTEGER, 
            lb=0, 
            name=f"x_{i}_{j}"
        )
    
    # Objective: Minimize sum of cost * x_ij
    model.setObjective(
        gp.quicksum(cost * x_vars[(i, j)] for (i, j, cost) in arcs),
        GRB.MINIMIZE
    )
    
    # 2.1. Required arcs constraint: x_ij >= 1 for (i,j) in required_arcs
    for (i, j, cost) in required_arcs:
        model.addConstr(x_vars[(i, j)] >= 1, name=f"req_{i}_{j}")
    
    # 2.2. Flow balance: for each node, sum in-flow = sum out-flow
    for v in nodes:
        in_flow = gp.quicksum(x_vars[(i, j)] 
                              for (i, j, c) in arcs 
                              if j == v)
        out_flow = gp.quicksum(x_vars[(i, j)]
                               for (i, j, c) in arcs
                               if i == v)
        model.addConstr(in_flow == out_flow, name=f"flow_bal_{v}")
    
    # 2.3. Integrality is already guaranteed by vtype=GRB.INTEGER
    
    # --- 3. Optimize ---
    model.optimize()
    
    # --- 4. Retrieve solution ---
    if model.status == GRB.OPTIMAL:
        print("\nOPTIMAL SOLUTION FOUND\n")
        print(f"Total Cost = {model.objVal}")
        
        # Print usage of arcs
        for (i, j, cost) in arcs:
            val = x_vars[(i, j)].X
            if val > 0.5:  # if used
                print(f"Arc ({i}->{j}), cost={cost}, traversed {int(val)} time(s)")
        
        model.write("Directed_Rural_Postman.lp")
    else:
        print(f"No optimal solution found (status={model.status})")

if __name__ == "__main__":
    solve_directed_rural_postman(required_arcs,optional_arcs)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 13 rows, 15 columns and 35 nonzeros
Model fingerprint: 0x0755b175
Variable types: 0 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 55.0000000


Presolve removed 9 rows and 8 columns
Presolve time: 0.00s
Presolved: 4 rows, 7 columns, 14 nonzeros
Variable types: 0 continuous, 7 integer (0 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0        55.00000   55.00000  0.00%     -    0s

Explored 1 nodes (2 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 55 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.500000000000e+01, best bound 5.500000000000e+01, gap 0.0000%

OPTIMAL SOLUTION FOUND

Total Cost = 55.0
Arc (1->2), cost=6, traversed 1 time(s)
Arc (3->1), cost=9, traversed 1 time(s)
Arc (2->5), cost=7, traversed 1 time(s)
Arc (6->4), cost=8, traversed 1 time(s)
Arc (7->8), cost=7, traversed 1 time(s)
Arc (4->3), cost=3, traversed 1 time(s)